In [ ]:
!pip install transformers jsonlines

In [2]:
import warnings

warnings.filterwarnings('ignore')

In [3]:
import json
import jsonlines
import numpy as np
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import functools
from collections import Counter
from argparse import Namespace
from transformers import BertTokenizer, BertModel
from tqdm.notebook import tqdm
import re
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import shutil
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import requests

In [ ]:
import train_functions as tf
import get_datasets as gd

In [10]:
image_encoders_dict = {
    'resnet18': 512,
    'resnet50': 2048,
    'resnet152': 2048
}

In [11]:
text_encoders_dict = {
    'prajjwal1/bert-mini': 128,
    'bert-base-uncased': 768,
    'bert-large-uncased': 1024
}

In [34]:
args = Namespace()

args.savedir_multimodal = os.path.join('model_save', 'multimodal')
args.savedir_text = os.path.join('model_save', 'text')
args.savedir_image = os.path.join('model_save', 'image')
args.data_path_mmimdb= 'mmimdb'
args.data_path_coco= 'coco'

args.train_perc = 0.8
args.val_perc = 0.1
args.max_seq_len = 512
args.batch_sz = 4
args.n_workers = 2
args.img_embed_pool_type = 'avg'
args.num_image_embeds = 1
args.lr = 1e-5
args.lr_patience = 2
args.lr_factor = 0.5
args.max_epochs = 5
args.gradient_accumulation_steps = 16
args.patience = 10

In [13]:
os.makedirs(args.savedir_multimodal, exist_ok=True)
os.makedirs(args.savedir_text, exist_ok=True)
os.makedirs(args.savedir_image, exist_ok=True)
os.makedirs(args.data_path_mmimdb, exist_ok=True)
os.makedirs(f'{args.data_path_mmimdb}/dataset', exist_ok=True)
os.makedirs(args.data_path_coco, exist_ok=True)
os.makedirs(f'{args.data_path_coco}/dataset', exist_ok=True)

In [21]:
captions_url = 'https://drive.usercontent.google.com/u/0/uc?id=1x_Tr6C52Nt3Oq7ZBYsBnivwUrq-PhzaD&export=download'
instances_url = 'https://drive.usercontent.google.com/u/0/uc?id=1_tW8EpxC1ffyjWy3hruZGJZXUlcxE-Me&export=download'

'''
captions_url = 'https://drive.usercontent.google.com/u/0/uc?id=1onVMlzXqYmgmf-xp6jcu1zm5RVBR6UKp&export=download'
instances_url = 'https://drive.usercontent.google.com/u/0/uc?id=1p-AiOlPMIiE0mg7u6lLMjtpQTq1PWdtc&export=download'
'''

gd.write_formated_data_coco(args.data_path_coco, captions_url, instances_url)

  0%|          | 0/100 [00:00<?, ?it/s]

In [15]:
dataset_url = 'https://giohmg.am.files.1drv.com/y4mVMnd-_Cq2OqbN2YyYKBclz_PTvLBhmP1WICms-Dpmtj_gTCln0tbh-28SxECL1LE34uNcBRo2wu_7bLHaaBBqN1P990Nlw47RH-Loq8S2Sz8XpmPTWTuQfDJ0Gj5ZNUeDkq_cRLqTqHQCx51V-VHKJ1jHz1gDBFCCsB0xMgAPdRFJgRDR6CnjAyJjRighF7l_FnbcgHjAypnxwfsJomGYA'
file_name = 'mmimdb_3000.tar.gz'

'''
dataset_url = 'https://archive.org/download/mmimdb/mmimdb.tar.gz'
file_name = 'mmimdb.tar.gz'
'''


gd.write_formated_data_mmimdb(args.data_path_mmimdb, dataset_url, file_name)

In [ ]:
args.bert_type = 'bert-large-uncased' # 'prajjwal1/bert-mini' / 'bert-base-uncased'
args.text_hidden_sz = text_encoders_dict[args.bert_type]

args.resnet_type = 'resnet18' # 'resnet50' / 'resnet152'
args.img_hidden_sz = image_encoders_dict[args.resnet_type]

args.linear_layer_dim = 100
args.linear_layer_count = 0

model_type, params_count, test_f1 = tf.main(args, args.data_path_mmimdb)